In [1]:
import pandas as pd
import re

# Tách dữ liệu cho mô hình Content-based Filtering

In [2]:
column_game = ["name", "details", "tags", "description"]
combined_data_game = pd.read_csv("data/steam.csv", usecols= column_game).dropna().drop_duplicates()
combined_data_game = combined_data_game.iloc[1:] 
combined_data_game.info()
combined_data_game.head()

FileNotFoundError: [Errno 2] No such file or directory: 'data/steam.csv'

# Content_base

In [3]:
combined_data_game = pd.read_csv("data/process_data/steam_games.csv", encoding="UTF-8")

In [4]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np
# Tạo một vectorizer TF-IDF cho các đặc trưng nội dung
tfidf_vectorizer = TfidfVectorizer(analyzer='word', ngram_range=(1, 3), min_df=0, stop_words='english')

# Tạo ma trận TF-IDF từ các đặc trưng nội dung (ví dụ: name, tag, details,description)
tfidf_matrix = tfidf_vectorizer.fit_transform(combined_data_game['name'] + ' ' + combined_data_game['tags']+ ' ' + combined_data_game['details'] + ' ' + combined_data_game['description'])

# Thiết lập kích thước chunk cho việc xử lý ma trận TF-IDF
chunk_size = 800

# Lấy số lượng chunk
num_chunks = tfidf_matrix.shape[0] // chunk_size + 1

# Khởi tạo ma trận tương tự cosine
cosine_similarities = np.zeros((tfidf_matrix.shape[0], tfidf_matrix.shape[0]), dtype=np.float32)
#cosine_similarities = cosine_similarity(tfidf_matrix, tfidf_matrix)

# Tính toán ma trận tương tự cosine theo từng phần
for i in range(num_chunks):
    start_idx = i * chunk_size
    end_idx = min((i + 1) * chunk_size, tfidf_matrix.shape[0])

    # Lấy đoạn ma trận TF-IDF
    chunk_matrix = tfidf_matrix[start_idx:end_idx]

    # Tính độ tương tự cho từng đoạn
    chunk_cosine_similarities = cosine_similarity(chunk_matrix, tfidf_matrix)

    # Cập nhật ma trận độ tương đồng cosine bằng các giá trị của đoạn con
    cosine_similarities[start_idx:end_idx, :] = chunk_cosine_similarities*50



ValueError: np.nan is an invalid document, expected byte or unicode string.

In [62]:
# Lấy top 5 trò chơi được đề xuất cho một trò chơi cụ thể
game_name = "Dota 2"
game_index = combined_data_game[combined_data_game['name'] == game_name].index[0]
game_index # index của game

27

In [59]:
for x in enumerate(cosine_similarities[game_index]):
    print(x[0], x[1])
    break

0 0.014177661


In [116]:
rating = 4
# Lấy danh sách các trò chơi được gợi ý có độ tương đồng giảm dần
similar_indices = sorted(enumerate(cosine_similarities[game_index]), key=lambda x: x[1], reverse=True)
recommended_games = [combined_data_game['name'].iloc[i[0]] for i in similar_indices[1:6]]
print(f'Các trò chơi được đề xuất cho game {game_name}: {recommended_games}')
print(type(recommended_games))

Các trò chơi được đề xuất cho game Dota 2: ['The International 2019 Battle Pass', 'The International 2019 Battle Pass - Level 50', 'The International 2019 Battle Pass - Level 100', '王者英雄 The Great Hero', 'Prime World']
<class 'list'>


In [129]:
top_n = 5
rating = 3
filter_similar_indices = []
for _, similar in similar_indices:
    similar = round(similar,1)
    if  similar >= rating and similar != 50:
        filter_similar_indices.append((_,similar))
recommended_games_data = combined_data_game.copy().iloc[[i[0] for i in filter_similar_indices[:top_n]]]
recommended_games_data["rating"] = [i[1] for i in filter_similar_indices[:top_n]]


for _, data in recommended_games_data.iterrows():
    print(f"Name: {data['name']} \nTags: {data['rating']}\n")

Name: The International 2019 Battle Pass 
Tags: 4.599999904632568

Name: The International 2019 Battle Pass - Level 50 
Tags: 4.0

Name: The International 2019 Battle Pass - Level 100 
Tags: 4.0



In [8]:


print("=============================")
print(f"Thuộc tính của game đề xuất {game_name}:")
game_data = combined_data_game.iloc[game_index]
print("Name:", game_data['name'])
print("Tags:", game_data['tags'])
print("=============================")



Thuộc tính của game đề xuất Dota 2:
Name: Dota 2
Tags: Free to Play,MOBA,Multiplayer,Strategy,e-sports,Team-Based,Competitive,Action,Online Co-Op,PvP,Difficult,Co-op,RTS,Tower Defense,Fantasy,RPG,Character Customization,Replay Value,Action RPG,Simulation


Các game được đề xuất tương đồng với Dota 2 và thể loại: 
[Free to Play,MOBA,Multiplayer,Strategy,e-sports,Team-Based,Competitive,Action,Online Co-Op,PvP,Difficult,Co-op,RTS,Tower Defense,Fantasy,RPG,Character Customization,Replay Value,Action RPG,Simulation]
Name: The International 2019 Battle Pass 
Tags: Action,Strategy,Free to Play

Name: The International 2019 Battle Pass - Level 50 
Tags: Action,Strategy,Free to Play

Name: The International 2019 Battle Pass - Level 100 
Tags: Free to Play,Strategy,Action

Name: 王者英雄 The Great Hero 
Tags: Early Access,Early Access,Strategy,RPG,Action,MOBA,Multiplayer,Indie,Singleplayer

Name: Prime World 
Tags: Free to Play,MOBA,Strategy,Multiplayer,RPG,Massively Multiplayer,Action,Fantasy,Team-Based,MMORPG,Co-op,PvP,Online Co-Op,Competitive,RTS,Replay Value,Action RPG,Steampunk,e-sports,Character Customization



In [43]:
print(recommended_games_data.head())

                                                 name  \
7240               The International 2019 Battle Pass   
26246   The International 2019 Battle Pass - Level 50   
19948  The International 2019 Battle Pass - Level 100   
34245                             王者英雄 The Great Hero   
23286                                     Prime World   

                                                    tags  \
7240                        Action,Strategy,Free to Play   
26246                       Action,Strategy,Free to Play   
19948                       Free to Play,Strategy,Action   
34245  Early Access,Early Access,Strategy,RPG,Action,...   
23286  Free to Play,MOBA,Strategy,Multiplayer,RPG,Mas...   

                                                 details  \
7240   Multi-player,Co-op,Downloadable Content,Steam ...   
26246  Multi-player,Co-op,Downloadable Content,Steam ...   
19948  Multi-player,Co-op,Downloadable Content,Steam ...   
34245  Single-player,Multi-player,Co-op,In-App Purcha...

# UNIQUE TAG & GAMENAME

In [ ]:
column_gametag = ["name", "tags"]
combined_data_game_tag = pd.read_csv("steam.csv", usecols= column_gametag).dropna().drop_duplicates(subset='name')
combined_data_game_tag =combined_data_game_tag.iloc[1:]
combined_data_game_tag.head(10)

In [ ]:

# Tách các giá trị trong cột "tags"
tags_split = combined_data_game_tag['tags'].str.split(",")

# Chuyển các giá trị thành các hàng riêng lẻ
tags_exploded = tags_split.explode()

# Lấy các giá trị duy nhất
unique_tags = tags_exploded.unique()

# Tạo DataFrame bảng chứa các tags duy nhất
tags_dataframe = pd.DataFrame(unique_tags, columns=['Unique Tags'])

# Tạo cột mới trong tags_dataframe để lưu danh sách tên game cho mỗi Unique Tag
tags_dataframe['Games'] = ""

# Loop qua từng Unique Tag và lấy danh sách tên game tương ứng
for index, row in tags_dataframe.iterrows():
    unique_tag = row['Unique Tags']
    games = combined_data_game_tag.loc[combined_data_game_tag['tags'].str.contains(unique_tag), 'name'].tolist()
    tags_dataframe.at[index, 'Games'] = ', '.join(games)

# In ra DataFrame bảng chứa các tags duy nhất và các game tương ứng
print(tags_dataframe)
tags_dataframe.to_csv('tag_game.csv', index=False)
